## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page <font color='red'> For Seller to update:[Title_of_your_product](Provide link to your marketplace listing of your product).</font>
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## Pipeline for National Cancer Institute Thesaurus (NCIt) Sentence Entity Resolver

- **Model**: `ncit_resolver_pipeline`
- **Model Description**: This advanced pipeline extracts oncological entities from clinical texts and utilizes the sbiobert_base_cased_mli Sentence Bert Embeddings to map these entities to their corresponding National Cancer Institute Thesaurus (NCIt) codes.

In [4]:
model_package_arn = "<Customer to specify Model package ARN corresponding to their AWS region>"

In [5]:
import base64
import json
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
import boto3
from IPython.display import Image, display
from PIL import Image as ImageEdit
import numpy as np

In [6]:
sagemaker_session = sage.Session()
s3_bucket = sagemaker_session.default_bucket()
region = sagemaker_session.boto_region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")
role = get_execution_role()

sagemaker = boto3.client("sagemaker")
s3_client = sagemaker_session.boto_session.client("s3")
ecr = boto3.client("ecr")
sm_runtime = boto3.client("sagemaker-runtime")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).

In [7]:
model_name = "ncit-resolver-pipeline"

real_time_inference_instance_type = "ml.m4.xlarge"
batch_transform_inference_instance_type = "ml.m4.xlarge"


### A. Create an endpoint

In [8]:
# create a deployable model from the model package.
model = ModelPackage(
    role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session
)

# Deploy the model
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

----------!

Once endpoint has been created, you would be able to perform real-time inference.

In [9]:
import json
import pandas as pd
import os
import boto3

# Set display options
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

def process_data_and_invoke_realtime_endpoint(data, content_type, accept):

    content_type_to_format = {'application/json': 'json', 'application/jsonlines': 'jsonl'}
    input_format = content_type_to_format.get(content_type)
    if content_type not in content_type_to_format.keys() or accept not in content_type_to_format.keys():
        raise ValueError("Invalid content_type or accept. It should be either 'application/json' or 'application/jsonlines'.")

    i = 1
    input_dir = f'inputs/real-time/{input_format}'
    output_dir = f'outputs/real-time/{input_format}'
    s3_input_dir = f"{model_name}/validation-input/real-time/{input_format}"
    s3_output_dir = f"{model_name}/validation-output/real-time/{input_format}"

    input_file_name = f'{input_dir}/input{i}.{input_format}'
    output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    while os.path.exists(input_file_name) or os.path.exists(output_file_name):
        i += 1
        input_file_name = f'{input_dir}/input{i}.{input_format}'
        output_file_name = f'{output_dir}/{os.path.basename(input_file_name)}.out'

    os.makedirs(os.path.dirname(input_file_name), exist_ok=True)
    os.makedirs(os.path.dirname(output_file_name), exist_ok=True)

    input_data = json.dumps(data) if content_type == 'application/json' else data

    # Write input data to file
    with open(input_file_name, 'w') as f:
        f.write(input_data)

    # Upload input data to S3
    s3_client.put_object(Bucket=s3_bucket, Key=f"{s3_input_dir}/{os.path.basename(input_file_name)}", Body=bytes(input_data.encode('UTF-8')))

    # Invoke the SageMaker endpoint
    response = sm_runtime.invoke_endpoint(
        EndpointName=model_name,
        ContentType=content_type,
        Accept=accept,
        Body=input_data,
    )

    # Read response data
    response_data = json.loads(response["Body"].read().decode("utf-8")) if accept == 'application/json' else response['Body'].read().decode('utf-8')

    # Save response data to file
    with open(output_file_name, 'w') as f_out:
        if accept == 'application/json':
            json.dump(response_data, f_out, indent=4)
        else:
            for item in response_data.split('\n'):
                f_out.write(item + '\n')

    # Upload response data to S3
    output_s3_key = f"{s3_output_dir}/{os.path.basename(output_file_name)}"
    if accept == 'application/json':
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=json.dumps(response_data).encode('UTF-8'))
    else:
        s3_client.put_object(Bucket=s3_bucket, Key=output_s3_key, Body=response_data)

    return response_data

### Initial Setup

In [10]:
docs = [
    '''A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.''',
    '''She underwent a computed tomography (CT) scan of the abdomen and pelvis, which showed a complex ovarian mass. A Pap smear performed one month later was positive for atypical glandular cells suspicious for adenocarcinoma. The pathologic specimen showed extension of the tumor throughout the fallopian tubes, appendix, omentum, and 5 out of 5 enlarged lymph nodes. The final pathologic diagnosis of the tumor was stage IIIC papillary serous ovarian adenocarcinoma. Two months later, the patient was diagnosed with lung metastases.'''
]


sample_text = '''There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.'''

### JSON

#### Example 1

  **Input format**:
  
  
```json
{
    "text": "Single text document"
}
```

In [11]:
input_json_data = {"text": sample_text}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,ncit_code,ncit_resolution,ncit_confidence
0,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",adenocarcinoma,56,69,Cancer_Dx,0.9998,C27813,adenocarcinoma [adenocarcinoma],0.9157
1,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",acinar cell,72,82,Histological_Type,0.98889995,C13077,acinar cell [acinar cell],0.9381
2,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",carcinoma,84,92,Cancer_Dx,0.9992,C2916,carcinoma [carcinoma],0.9366
3,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",hepatoid,97,104,Histological_Type,0.9959,C79753,hepatocellular [hepatocellular],0.1771
4,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",carcinoma of the pancreas,106,130,Cancer_Dx,0.86300004,C3850,carcinoma of the pancreas [carcinoma of the pancreas],0.8888
5,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",neoplastic infiltration,172,194,Invasion,0.63014996,C3262,neoplastic growth [neoplastic growth],0.1609
6,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",perineural,222,231,Site_Other_Body_Part,0.8878,C25615,perineural [perineural],0.9703
7,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",invasion,233,240,Invasion,0.9605,C20625,invasion [invasion],0.9925
8,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",lymph node,246,255,Site_Lymph_Node,0.88265,C12745,lymph node [lymph node],0.9066
9,"There was no differentiated component characteristic of adenocarcinoma, acinar cell carcinoma or hepatoid carcinoma of the pancreas. The surgical margins were negative for neoplastic infiltration. There was no evidence of perineural invasion. No lymph node metastasis was shown.",metastasis,257,266,Metastasis,0.9999,C19151,metastasis [metastasis],0.8954


#### Example 2

  **Input format**:
  
  
```json
{
    "text": [
        "Text document 1",
        "Text document 2",
        ...
    ]
}
```

In [12]:
input_json_data = {"text": docs}

data =  process_data_and_invoke_realtime_endpoint(input_json_data, content_type="application/json" , accept="application/json" )
pd.DataFrame(data["predictions"])

,document,ner_chunk,begin,end,ner_label,ner_confidence,ncit_code,ncit_resolution,ncit_confidence
0,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",debulking surgery,37,53,Cancer_Surgery,0.78400004,C15749,debulking surgery [debulking surgery],0.9964
1,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",bilateral oophorectomy,56,77,Cancer_Surgery,0.83615005,C51590,bilateral oophorectomy [bilateral oophorectomy],0.9848
2,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",omentectomy,84,94,Cancer_Surgery,0.988,C51787,omentectomy [omentectomy],0.9934
3,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",total hysterectomy,97,114,Cancer_Surgery,0.95624995,C15701,total hysterectomy [total hysterectomy],0.9922
4,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",radical pelvic lymph nodes dissection,121,157,Cancer_Surgery,0.80143994,C48936,radical lymph node dissection [radical lymph node dissection],0.3509
5,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",ovarian carcinoma,166,182,Cancer_Dx,0.92355,C4908,ovarian carcinoma [ovarian carcinoma],0.9342
6,"A 65-year-old

### JSON Lines

In [13]:
import json

def create_jsonl(records):
    json_records = []

    for text in records:
        record = {
            "text": text
        }
        json_records.append(record)

    json_lines = '\n'.join(json.dumps(record) for record in json_records)

    return json_lines

input_jsonl_data = create_jsonl(docs)

#### Example 1

  **Input format**:
  
```json
{"text": "Text document 1"}
{"text": "Text document 2"}
```

In [14]:
data = process_data_and_invoke_realtime_endpoint(input_jsonl_data, content_type="application/jsonlines" , accept="application/jsonlines" )
print(data)

{"predictions": [{"ner_chunk": "debulking surgery", "begin": 37, "end": 53, "ner_label": "Cancer_Surgery", "ner_confidence": "0.78400004", "ncit_code": "C15749", "ncit_resolution": "debulking surgery [debulking surgery]", "ncit_confidence": "0.9964"}, {"ner_chunk": "bilateral oophorectomy", "begin": 56, "end": 77, "ner_label": "Cancer_Surgery", "ner_confidence": "0.83615005", "ncit_code": "C51590", "ncit_resolution": "bilateral oophorectomy [bilateral oophorectomy]", "ncit_confidence": "0.9848"}, {"ner_chunk": "omentectomy", "begin": 84, "end": 94, "ner_label": "Cancer_Surgery", "ner_confidence": "0.988", "ncit_code": "C51787", "ncit_resolution": "omentectomy [omentectomy]", "ncit_confidence": "0.9934"}, {"ner_chunk": "total hysterectomy", "begin": 97, "end": 114, "ner_label": "Cancer_Surgery", "ner_confidence": "0.95624995", "ncit_code": "C15701", "ncit_resolution": "total hysterectomy [total hysterectomy]", "ncit_confidence": "0.9922"}, {"ner_chunk": "radical pelvic lymph nodes disse

### C. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [15]:
model.sagemaker_session.delete_endpoint(model_name)
model.sagemaker_session.delete_endpoint_config(model_name)

## 3. Batch inference

In [16]:
import json
import os

input_dir = 'inputs/batch'
json_input_dir = f"{input_dir}/json"
jsonl_input_dir = f"{input_dir}/jsonl"

output_dir = 'outputs/batch'
json_output_dir = f"{output_dir}/json"
jsonl_output_dir = f"{output_dir}/jsonl"

os.makedirs(json_input_dir, exist_ok=True)
os.makedirs(jsonl_input_dir, exist_ok=True)
os.makedirs(json_output_dir, exist_ok=True)
os.makedirs(jsonl_output_dir, exist_ok=True)

validation_json_file_name = "input.json"

validation_jsonl_file_name = "input.jsonl"

validation_input_json_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/json/"
validation_output_json_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/json/"

validation_input_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-input/batch/jsonl/"
validation_output_jsonl_path = f"s3://{s3_bucket}/{model_name}/validation-output/batch/jsonl/"

def write_and_upload_to_s3(input_data, file_name):
    file_format = os.path.splitext(file_name)[1].lower()
    if file_format == ".json":
        input_data = json.dumps(input_data)

    with open(file_name, "w") as f:
        f.write(input_data)

    s3_client.put_object(
        Bucket=s3_bucket,
        Key=f"{model_name}/validation-input/batch/{file_format[1:]}/{os.path.basename(file_name)}",
        Body=(bytes(input_data.encode("UTF-8"))),
    )

In [17]:
input_jsonl_data = create_jsonl(docs)
input_json_data = {"text": docs}

write_and_upload_to_s3(input_json_data, f"{json_input_dir}/{validation_json_file_name}")

write_and_upload_to_s3(input_jsonl_data, f"{jsonl_input_dir}/{validation_jsonl_file_name}")

### JSON

In [ ]:
# Initialize a SageMaker Transformer object for making predictions
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/json",
    output_path=validation_output_json_path
)

transformer.transform(validation_input_json_path, content_type="application/json")
transformer.wait()

In [19]:
from urllib.parse import urlparse

def process_s3_json_output_and_save(validation_file_name):

    output_file_path = f"{json_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = json.loads(response["Body"].read().decode("utf-8"))
    df = pd.DataFrame(data["predictions"])
    display(df)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        json.dump(data, f_out, indent=4)

In [20]:
process_s3_json_output_and_save(validation_json_file_name)

,document,ner_chunk,begin,end,ner_label,ner_confidence,ncit_code,ncit_resolution,ncit_confidence
0,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",debulking surgery,37,53,Cancer_Surgery,0.78400004,C15749,debulking surgery [debulking surgery],0.9964
1,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",bilateral oophorectomy,56,77,Cancer_Surgery,0.83615005,C51590,bilateral oophorectomy [bilateral oophorectomy],0.9848
2,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",omentectomy,84,94,Cancer_Surgery,0.988,C51787,omentectomy [omentectomy],0.9934
3,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",total hysterectomy,97,114,Cancer_Surgery,0.95624995,C15701,total hysterectomy [total hysterectomy],0.9922
4,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",radical pelvic lymph nodes dissection,121,157,Cancer_Surgery,0.80143994,C48936,radical lymph node dissection [radical lymph node dissection],0.3509
5,"A 65-year-old woman had a history of debulking surgery, bilateral oophorectomy with omentectomy, total hysterectomy with radical pelvic lymph nodes dissection due to ovarian carcinoma (mucinous-type carcinoma, stage Ic) 1 year ago. Patient's medical compliance was poor and failed to complete her chemotherapy (cyclophosphamide 750 mg/m2, carboplatin 300 mg/m2). Recently, she noted a palpable right breast mass, 15 cm in size which nearly occupied the whole right breast in 2 months. Core needle biopsy revealed metaplastic carcinoma.",ovarian carcinoma,166,182,Cancer_Dx,0.92355,C4908,ovarian carcinoma [ovarian carcinoma],0.9342
6,"A 65-year-old

### JSON Lines

In [ ]:
transformer = model.transformer(
    instance_count=1,
    instance_type=batch_transform_inference_instance_type,
    accept="application/jsonlines",
    output_path=validation_output_jsonl_path
)
transformer.transform(validation_input_jsonl_path, content_type="application/jsonlines")
transformer.wait()

In [22]:
from urllib.parse import urlparse

def process_s3_jsonlines_output_and_save(validation_file_name):

    output_file_path = f"{jsonl_output_dir}/{validation_file_name}.out"
    parsed_url = urlparse(transformer.output_path)
    file_key = f"{parsed_url.path[1:]}{validation_file_name}.out"
    response = s3_client.get_object(Bucket=s3_bucket, Key=file_key)

    data = response["Body"].read().decode("utf-8")
    print(data)

    # Save the data to the output file
    with open(output_file_path, 'w') as f_out:
        for item in data.split('\n'):
            f_out.write(item + '\n')

In [23]:
process_s3_jsonlines_output_and_save(validation_jsonl_file_name)

{"predictions": [{"ner_chunk": "debulking surgery", "begin": 37, "end": 53, "ner_label": "Cancer_Surgery", "ner_confidence": "0.78400004", "ncit_code": "C15749", "ncit_resolution": "debulking surgery [debulking surgery]", "ncit_confidence": "0.9964"}, {"ner_chunk": "bilateral oophorectomy", "begin": 56, "end": 77, "ner_label": "Cancer_Surgery", "ner_confidence": "0.83615005", "ncit_code": "C51590", "ncit_resolution": "bilateral oophorectomy [bilateral oophorectomy]", "ncit_confidence": "0.9848"}, {"ner_chunk": "omentectomy", "begin": 84, "end": 94, "ner_label": "Cancer_Surgery", "ner_confidence": "0.988", "ncit_code": "C51787", "ncit_resolution": "omentectomy [omentectomy]", "ncit_confidence": "0.9934"}, {"ner_chunk": "total hysterectomy", "begin": 97, "end": 114, "ner_label": "Cancer_Surgery", "ner_confidence": "0.95624995", "ncit_code": "C15701", "ncit_resolution": "total hysterectomy [total hysterectomy]", "ncit_confidence": "0.9922"}, {"ner_chunk": "radical pelvic lymph nodes disse

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: ncit-resolver-pipeline-2024-06-06-12-16-29-087


### Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.

